In [42]:
import pandas as pd
import os
import torch
import torch.nn as nn
import numpy as np
from dataset.datasets import CustomDataset
from torch.utils.data import DataLoader
from model.tabnet import TabNet
from model.classification_network import TabNetClassifier
from model.sparce_loss import SparceLoss
from tqdm.notebook import tqdm

In [43]:
DATA_PATH = os.path.join(os.path.abspath("."), "data/encoded_dataset.csv")

In [44]:
df = pd.read_csv(DATA_PATH)

In [45]:
df.head()

,Unnamed: 0,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,salary,age_log,fnlwgt_log,capital_loss_log,hours_per_week_bins,native_country
0,0,3,0,12.0,1,0,5,1,0,0,1,3.891820,11.526039,7.551187,2,1
1,1,3,4,14.0,0,3,1,1,1,10520,1,3.784190,12.374743,0.000000,3,1
2,2,3,3,10.0,0,0,4,0,0,0,0,3.637586,11.474029,0.000000,2,1
3,3,4,6,15.0,1,8,0,2,1,0,1,3.637586,11.633788,0.000000,2,1
4,4,5,7,10.0,1,7,5,0,0,0,0,3.737670,11.318090,0.000000,3,1


In [92]:
df.salary.mean()

0.24187760643972456

In [46]:
categorical_columns = ['workclass', "education", "marital-status", "occupation", 'relationship',
                       'race', 'sex', 'hours_per_week_bins', 'native_country']
numerical_columns = ['age_log', 'fnlwgt_log', "education-num", 'capital-gain', 'capital_loss_log']
target = 'salary'

In [93]:
len(numerical_columns)

5

In [94]:
32-5

27

In [97]:
len(categorical_columns)*3

27

In [47]:
dataset = CustomDataset(df=df, numerical_columns=numerical_columns,
                       categorical_columns=categorical_columns, target=target)

In [83]:
dataloader = DataLoader(dataset, batch_size=512, num_workers=2, shuffle=True)

In [84]:
model = TabNetClassifier(
                n_output_classes=2,
                 n_classification_layer=2,
                num_unique_values_dict = dataset.num_unique_values_dict,
                 embedding_size = 5,
                 n_numerical_columns = len(numerical_columns),
                 n_shared_layers = 2,
                 n_decision_blocks = 2,
                 hidden_size = 512,
                 meaningful_part = 0.3,
                 n_decision_steps = 4,
                 virtual_batch_size = 128,
                 momentum = 0.98,
                 gamma = 1.5)

In [85]:
sparce_criterion = SparceLoss()
classification_critetion = nn.CrossEntropyLoss()

In [86]:
# output, masks = model(X_batch)

In [87]:
# loss = criterion(masks)

In [88]:
# loss

In [89]:
valid_loss = 0
for X_cat, X_num, y_batch in tqdm(dataloader):
    x_batch = [X_cat, X_num]
#     y_batch = y_batch.reshape(-1,1).long()
    output, masks = model(x_batch)
    clf_loss = classification_criterion(output, y_batch)
    sparce_loss = sparce_criterion(masks)
    loss = clf_loss + sparce_loss
    valid_loss += loss.item()
#     predict = (torch.sigmoid(output.cpu()) > 0.5).long()
#     correct_valid += (y_batch.cpu() == predict).sum()

  0%|          | 0/61 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [90]:
valid_loss = 0
correct_valid = 0
for X_cat, X_num, y_batch in tqdm(dataloader):
    x_batch = [X_cat, X_num]
    output, masks = model(x_batch)
    clf_loss = classification_criterion(output, y_batch)
    sparce_loss = sparce_criterion(masks)
    loss = clf_loss + sparce_loss
    valid_loss += loss.item()
    # predict = (torch.sigmoid(output.cpu()) > 0.5).long()
    predict = np.argmax(torch.softmax(output, dim=-1).cpu().detach().numpy())
    correct_valid += (y_batch.cpu() == predict).sum()

  0%|          | 0/61 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [91]:
!python train.py

  0%|          | 0/12 [00:00<?, ?it/s]
Epoch: 0, Train loss: 0.0015,  Valid loss: 0.0013
Epoch: 0, Train Accuracy: 0.0,  Valid Accuracy: 0.0
  0%|          | 0/48 [00:00<?, ?it/s]
^C
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "train.py", line 183, in <module>
    main()
  File "/home/dmitrii/SberDL/HW5/venv/lib/python3.8/site-packages/hydra/main.py", line 32, in decorated_main
    _run_hydra(
  File "/home/dmitrii/SberDL/HW5/venv/lib/python3.8/site-packages/hydra/_internal/u